In [1]:
import sys
sys.path.append("..")
from sklearn.model_selection import train_test_split
import numpy as np

from movingpose.estimator import neighbors
from movingpose.estimator import classifiers

from movingpose.preprocessing import moving_pose
from movingpose.preprocessing import kinect_skeleton_data

## Pickle multiview action data

In [ ]:
kinect_skeleton_data.pickle_dir("../pickle/multiview.p", "../ext/dataset/multiview_action/")

Pickling skeleton data...


## Load pickled multiview action data

In [ ]:
raw_data_dict = kinect_skeleton_data.load_pickle("../pickle/multiview.p")

## Format multiview action data

In [ ]:
X, labels = moving_pose.format_skeleton_data_dict(raw_data_dict)

## Perform basic verification that skeleton data is formatted correctly

In [ ]:
for action in X:
    for frame_num, pose in enumerate(action):
        assert pose.shape == (20, 10), f"{pose.shape} =/= (10, )"
        for i, descriptor in enumerate(pose):
            if i == 0:
                assert list(descriptor)[:-1] == [0, 0, 0, 0, 0, 0, 0, 0, 0]
            assert descriptor[-1] == frame_num, f"{descriptor[-1]} =/= {frame_num}"

## Create train/test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, random_state=42)

## Train Action Classifier with normalized training data

In [ ]:
best_score = sys.maxsize
best_action_classifier = None
for n_neighbors in range(5, 15, 5):
    for n_training_neighbors in range(10, 40, 10):
        for alpha in np.arange(0.2, 0.8, 0.1):
            for beta in np.arange(0.1, 0.5, 0.1):
                for kappa in range(3, 10, 1):
                    nearest_pose_estimator = neighbors.NearestPoses(
                        n_neighbors=n_neighbors,
                        n_training_neighbors=n_training_neighbors,
                        alpha=alpha,
                        beta=beta,
                        kappa=kappa
                    )
                    for theta in np.arange(0.2, 0.8, 0.05):
                        for n in range(5, 15, 1):
                            action_classifier = classifiers.ActionClassifier(
                                nearest_pose_estimator=nearest_pose_estimator,
                                theta=theta,
                                n=n
                            )
                            action_classifier.fit(X_train, y_train)
                            y_pred = action_classifier.predict(X_test)

                            diff = 0
                            for pred, actual in zip(y_pred, y_test):
                                diff += abs(pred - actual)

                            if diff < best_score:
                                best_score = diff
                                best_action_classifier = action_classifier

## Print best action classifier and its score

In [ ]:
print(f"The best action classifier was {best_action_classifier}")
print("\n--------------\n")
print(f"Its score was: {best_score}")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Predict live data

In [ ]:
#TODO add live predictions when mehmet publishes the UI code

# DAB

In [ ]:
print("dab")
